Ensambles de modelos
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/R-for-predictive-analytics/blob/master/18-ensambles.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/R-for-predictive-analytics/blob/master/18-ensambles.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/R-for-predictive-analytics/blob/master/LICENSE)  
[Readme](https://github.com/jdvelasq/R-for-predictive-analytics/blob/master/readme.md)

# Definición del problema real

Se desea construir un esquema de pronóstico que permita aprovechar la información capturada por muchos modelos diferentes construidos sobre los mismos datos.

# Definición del problema en términos de los datos

Si se tienen múltiples pronósticos obtenidos por diferentes modelos para los mismos datos, es mejor simplemente seleccionar el mejor modelo y descartar la demás información, o se pueden aprovechar dichos pronósticos?

# Solución

Un ensamble es un tipo de modelo que permite la combinación de varios modelos de predicción para obtener un solo pronóstico basado en los pronósticos individuales de cada modelo. En la figura siguiente se presenta un esquema ilustrativo.

![alt text](images/combiner.jpg)



La clave de la operación de esta metodología se basa en la diversidad, la cual puede obtenerse de diferentes formas:

* Variando los datos de entrenamiento: se usa el mismo modelo en todos los casos, pero para cada uno de ellos se usa una muestra de entrenamiento obtenida por boostraping; así cada modelo tiene parámetros diferentes ya que fue estimado sobre una muestra diferente.


* Variando la configuración del modelo: se usan exactamente los mismos datos de entrenamiento, pero sobre diferentes modelos obtenidos variando su configuración; por ejemplo, el mismo modelo pero con diferente configuración (entradas usadas, complejidad, etc). Inclusive se pueden utilizar distintos modelos.


* Una combinación de los dos anteriores.

El combinador es un mecanismo que obtiene un único pronóstico a partir de los pronósticos individuales de cada modelo. Para problemas de clasificación, la conbinación se hace por votación. En problemas de regresión, mediante promedio simple, promedio combinado o, inclusive, regresión.

Diferentes metodologías se han desarrollado sobre este concepto.

### Bagging (Bootstrap aggregation)

En esta metodología, la diversidad se obtiene al entrenar un mismo modelo sobre diferentes conjuntos de entrenamiento usando bootstraping. La combinación se hace por votación para problemas de clasificación y por promedio para problemas numéricos.

In [1]:
## install.packages("ipred")
library(ipred)

In [2]:
##
## crea 25 arboles de decisión
##
data <- read.csv("data/credit.csv")
data$default <- factor(data$default, labels=c("No", "Yes"))
mybag <- bagging(default ~ ., data = data, nbagg = 25)
data_pred <- predict(mybag, data)
table(data_pred, data$default)

         
data_pred  No Yes
      No  700   6
      Yes   0 294

In [3]:
##
## Para evaluar realmente la precisión del modelo
## se usa crossvalidation
##
library(caret)
set.seed(300)
ctrl <- trainControl(method = "cv", number = 10)
train(default ~ ., data = data, method = "treebag", trControl = ctrl)
# note que tiene un desempeño muy similar al metodo C5.0

Loading required package: lattice
Loading required package: ggplot2


Bagged CART 

1000 samples
  20 predictor
   2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results:

  Accuracy  Kappa  
  0.746     0.36038


### AdaBoosting (Adaptive Boosting)

En este caso, los conjuntos de dato son diseñados especificamente para generar modelos complementarios. De forma simplificada, el algoritmo procede de la siguiente forma:

* Paso 1: Se construye un clasificador sobre todos los datos de entrenamiento.


* Paso 2: Se construye un nuevo conjunto de datos con los ejemplos mal clasificados (o una porción de ellos).


* Paso 3: se construye un nuevo clasificador con los datos obtenidos en el paso 2.


* Paso 4: Se retorna al Paso 2.


El proceso itera hasta que se alcanza una precisión requerida o el número máximo de clasificadores permitidos en el ensamble. La ponderación se realiza dando más peso a los modelos con mejor desempeño, de tal forma, que el desempeño es, al menos, similar al del mejor clasificador obtenido.

In [5]:
## install.packages("adabag")
library(adabag)
set.seed(300)
m_adaboost <- boosting(default ~ ., data = data)
p_adaboost <- predict(m_adaboost, data)
p_adaboost$confusion

Loading required package: rpart
Loading required package: foreach
Loading required package: doParallel
Loading required package: iterators
Loading required package: parallel

Attaching package: ‘adabag’

The following object is masked from ‘package:ipred’:

    bagging



               Observed Class
Predicted Class  No Yes
            No  700   0
            Yes   0 300

In [6]:
##
## verificación usando CV
##
set.seed(300)
adaboost_cv <- boosting.cv(default ~ ., data = data)
adaboost_cv$confusion

i:  1 Thu Mar 15 20:42:27 2018 
i:  2 Thu Mar 15 20:43:03 2018 
i:  3 Thu Mar 15 20:43:38 2018 
i:  4 Thu Mar 15 20:44:15 2018 
i:  5 Thu Mar 15 20:44:52 2018 
i:  6 Thu Mar 15 20:45:27 2018 
i:  7 Thu Mar 15 20:46:02 2018 
i:  8 Thu Mar 15 20:46:38 2018 
i:  9 Thu Mar 15 20:47:13 2018 
i:  10 Thu Mar 15 20:47:48 2018 


               Observed Class
Predicted Class  No Yes
            No  594 153
            Yes 106 147

In [7]:
## install.packages("vcd")
library(vcd)
Kappa(adaboost_cv$confusion)

Loading required package: grid


            value     ASE     z Pr(>|z|)
Unweighted 0.3544 0.03237 10.95 6.68e-28
Weighted   0.3544 0.03237 10.95 6.68e-28

### Random forest

En este método se combinan el bagging con la selección aleatoria de caracteríticas para aumetar la diversidad. La salida del modelo se obtiene por votación. Cada ejemplo que no es considerado durante el entrenamiento es usado como parte del conjunto de prueba.

In [8]:
## install.packages("randomForest")
library(randomForest)
set.seed(300)

##
## por defecto crea un ensamble de 500 arboles
## que usan la raíz cuadrada de la cantidad
## de atributos presentes en el conjunto de entrenamiento
##
rf <- randomForest(default ~ ., data = data)
rf
## en la salida OOB se refiere a out-of-bag error rate

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin




Call:
 randomForest(formula = default ~ ., data = data) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 4

        OOB estimate of  error rate: 22.7%
Confusion matrix:
     No Yes class.error
No  649  51  0.07285714
Yes 176 124  0.58666667

In [9]:
##
## A continuación se comparan los resultados de 
## random forest (rf) con C5.0 
##
library(caret)

##
## parametros de control
##
ctrl <- trainControl(method = "repeatedcv",
                     number = 10, 
                     repeats = 10)

##
## malla de parametros a considerar
##
grid_rf <- expand.grid(.mtry = c(2, 4, 8, 16))

set.seed(300)
m_rf <- train(default ~ ., 
              data = data, 
              method = "rf",
              metric = "Kappa", 
              trControl = ctrl,
              tuneGrid = grid_rf)
m_rf 

Random Forest 

1000 samples
  20 predictor
   2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  mtry  Accuracy  Kappa    
   2    0.7191    0.0990317
   4    0.7491    0.2813675
   8    0.7546    0.3294882
  16    0.7562    0.3527924

Kappa was used to select the optimal model using the largest value.
The final value used for the model was mtry = 16.

In [10]:
##
## comparación con un boosted tree
##
grid_c50 <- expand.grid(.model = "tree",
                        .trials = c(10, 20, 30, 40),
                        .winnow = "FALSE")
set.seed(300)
m_c50 <- train(default ~ ., 
               data = data, 
               method = "C5.0",
               metric = "Kappa", 
               trControl = ctrl,
               tuneGrid = grid_c50)
m_c50

Warning message in Ops.factor(x$winnow):
“‘!’ not meaningful for factors”

C5.0 

1000 samples
  20 predictor
   2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  trials  Accuracy  Kappa    
  10      0.7404    0.3389905
  20      0.7476    0.3557574
  30      0.7502    0.3623037
  40      0.7509    0.3645426

Tuning parameter 'model' was held constant at a value of tree
Tuning
 parameter 'winnow' was held constant at a value of FALSE
Kappa was used to select the optimal model using the largest value.
The final values used for the model were trials = 40, model = tree and winnow
 = FALSE.

**Ejercicio.--** Cuál de los dos modelos anteriores fue mejor?

---

Ensambles de modelos
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/R-for-predictive-analytics/blob/master/18-ensambles.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/R-for-predictive-analytics/blob/master/18-ensambles.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/R-for-predictive-analytics/blob/master/LICENSE)  
[Readme](https://github.com/jdvelasq/R-for-predictive-analytics/blob/master/readme.md)